<a href="https://colab.research.google.com/github/Ambica06/EncryptionForLinearRegression/blob/main/Encryption_For_ML_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import warnings
warnings.simplefilter("ignore")

import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
file = ('kc_house_data.csv')

df = pd.read_csv(file)

if not df.empty:
    print('Dataframe and libraries imported successfully!')

Dataframe and libraries imported successfully!


In [ ]:
df_train, df_test = train_test_split(df, test_size = 0.33, random_state = 41, stratify=df['zipcode'])

df_grouped = df_train.groupby(by='zipcode').agg(region_price=('price', 'mean')).reset_index()
df_grouped['quartile_zone'] = pd.qcut(df_grouped['region_price'], q=4, labels=[1, 2, 3, 4])

df_train = df_train.merge(df_grouped[['zipcode', 'quartile_zone']], how='left', on='zipcode')
df_test = df_test.merge(df_grouped[['zipcode', 'quartile_zone']], how='left', on='zipcode')
df_train.drop(columns=['zipcode'], inplace=True)
df_test.drop(columns=['zipcode'], inplace=True)

In [ ]:
X_train_raw = df_train.drop(columns=['date', 'price', 'bedrooms'])
X_test_raw = df_test.drop(columns=['date', 'price', 'bedrooms'])
y_train = df_train['price']
y_test = df_test['price']

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train_raw)

X_train = pd.DataFrame(scaler.transform(X_train_raw), columns = X_train_raw.columns)
X_train.set_index(X_train_raw.index, inplace = True)

X_test = pd.DataFrame(scaler.transform(X_test_raw), columns = X_test_raw.columns)
X_test.set_index(X_test_raw.index, inplace = True)

In [ ]:
model_1 = sm.OLS(y_train, sm.add_constant(X_train)).fit()
print(model_1.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.744
Model:                            OLS   Adj. R-squared:                  0.743
Method:                 Least Squares   F-statistic:                     2467.
Date:                Mon, 25 Nov 2024   Prob (F-statistic):               0.00
Time:                        21:30:22   Log-Likelihood:            -1.9615e+05
No. Observations:               14480   AIC:                         3.923e+05
Df Residuals:                   14462   BIC:                         3.925e+05
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          5.409e+05   1537.128    351.910

In [ ]:
vif = pd.DataFrame({'feature':X_train.columns})
vif['VIF'] = [variance_inflation_factor((X_train * 1).values, i) for i in range(X_train.shape[1])]
vif.sort_values(by= 'VIF', ascending=False)

,feature,VIF
9,sqft_above,inf
2,sqft_living,inf
10,sqft_basement,inf
8,grade,3.571498
1,bathrooms,3.226099
15,sqft_living15,3.021022
11,yr_built,2.456862
17,quartile_zone,2.187051
16,sqft_lot15,2.150149
3,sqft_lot,2.135603


In [ ]:
model_robust = model_1.get_robustcov_results()

In [ ]:
pred = model_robust.predict(sm.add_constant(X_test))

print(f'R² score: {r2_score(y_test, pred)*100}')

print("Root Mean Squared Error:",np.sqrt(mean_squared_error(y_test, pred)))

print("Mean Absolute Error:",mean_absolute_error(y_test, pred))

print("Mean Absolute Percentage Error:",np.mean(np.abs( (y_test-pred) / y_test))*100)

R² score: 71.99067998217855
Root Mean Squared Error: 196514.05996866943
Mean Absolute Error: 111977.68487313674
Mean Absolute Percentage Error: 22.774365139204214
